In [ ]:
import os, sys

import numpy as np
import pandas as pd
import geopandas as gpd

import fct_misc

from tqdm import tqdm


# Official cadastral survey


## Read data


In [ ]:
INITIAL_FOLDER = "/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/initial"
PROCESSED_FOLDER = "/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed"

ROADS_CS = "MO/cleaned_roads_MO.gpkg"
ROADS_LINES = "TLM/Strassen/corrected_roads_inside_AOI.shp"
# ROADS_TLM = "shapefiles_gpkg/roads_for_OD.shp"
GT_ROADS="json/ground_truth_labels.geojson"
OTH_ROADS="json/other_labels.geojson"
FOREST = "TLM/Clip_Wald_Bodenbedeckung/forests.shp"

ROADS_PARAM = "TLM/Strassen/Objektarten_to_keep.xlsx"

NOT_ROAD = [12, 13, 14, 19, 22, 23]
KUNSTBAUTE_TO_KEEP = [100, 200]


In [ ]:
roads_cs_good = gpd.read_file(os.path.join(
    INITIAL_FOLDER, ROADS_CS), layer="good_roads_cleaned")
roads_cs_bad = gpd.read_file(os.path.join(
    INITIAL_FOLDER, ROADS_CS), layer="bad_roads_cleaned")
# roads_tlm = gpd.read_file(os.path.join(PROCESSED_FOLDER, ROADS_TLM))
gt_roads=gpd.read_file(os.path.join(PROCESSED_FOLDER, GT_ROADS))
oth_roads=gpd.read_file(os.path.join(PROCESSED_FOLDER, OTH_ROADS))

roads_lines = gpd.read_file(os.path.join(INITIAL_FOLDER, ROADS_LINES))
forest = gpd.read_file(os.path.join(INITIAL_FOLDER, FOREST))

roads_param = pd.read_excel(os.path.join(INITIAL_FOLDER, ROADS_PARAM))


In [ ]:
roads_cs = pd.concat([roads_cs_good, roads_cs_bad])
roads_tlm = pd.concat([gt_roads, oth_roads])


## Treat data


Make the joined domain


In [ ]:
roads_cs=roads_cs.to_crs(crs=roads_tlm.crs)
roads_cs = fct_misc.test_valid_geom(roads_cs, correct=True, gdf_obj_name='roads from the cadastral survey')

# roads_cs = fct_misc.test_valid_geom(roads_cs, correct=False, gdf_obj_name='roads from the cadastral survey')


In [ ]:
roads_tlm=fct_misc.test_valid_geom(roads_tlm, correct=False, gdf_obj_name='roads from the TLM')

In [ ]:
fct_misc.test_crs(roads_cs.crs, roads_tlm.crs)
roads_domain = gpd.overlay(roads_tlm, roads_cs, how="union")


In [ ]:
temp = roads_domain.unary_union
roads_domain_union = gpd.GeoDataFrame({'id': [k for k in range(len(temp.geoms))], 'geometry': [geom for geom in temp.geoms]},
                                      crs=roads_domain.crs)


In [ ]:
roads_domain_union.to_file(os.path.join(
    PROCESSED_FOLDER, "shapefiles_gpkg/test_MO_TLM.shp"))


Get roads parts


In [ ]:
roads_of_interest = roads_lines[~roads_lines['OBJEKTART'].isin(NOT_ROAD)]
uncovered_roads = roads_of_interest[roads_of_interest['KUNSTBAUTE'].isin(
    KUNSTBAUTE_TO_KEEP)]

roads_param_filtered = roads_param[~roads_param['Width'].isna()].copy()

# From observation, adjusting the road with to keep as much as the cadastral survey as possible
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 0, 'Width'] = 7
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 1, 'Width'] = 7
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 2, 'Width'] = 10
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 11, 'Width'] = 5
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 15, 'Width'] = 3
roads_param_filtered.loc[roads_param_filtered['GDB-Code'] == 21, 'Width'] = 8


roads_param_filtered.drop_duplicates(
    subset='GDB-Code', inplace=True)       # Keep first by default

uncovered_roads = uncovered_roads.merge(roads_param_filtered[[
                                        'GDB-Code', 'Width']], how='inner', left_on='OBJEKTART', right_on='GDB-Code')

uncovered_roads.drop(columns=[
    'DATUM_AEND', 'DATUM_ERST', 'ERSTELLUNG', 'ERSTELLU_1', 'UUID',
    'REVISION_J', 'REVISION_M', 'GRUND_AEND', 'HERKUNFT', 'HERKUNFT_J',
    'HERKUNFT_M', 'REVISION_Q', 'WANDERWEGE', 'VERKEHRSBE',
    'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG',
    'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME',
    'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng'], inplace=True)

buffered_roads = uncovered_roads.copy()
buffered_roads['geometry'] = uncovered_roads.buffer(
    uncovered_roads['Width'], cap_style=2)

# Do not let roundabout parts make artifacts
buff_geometries = []
for geom in buffered_roads['geometry'].values:
    if geom.geom_type == 'MultiPolygon':
        buff_geometries.append(max(geom.geoms, key=lambda a: a.area))
    else:
        buff_geometries.append(geom)

buffered_roads['geometry'] = buff_geometries

# Erase overlapping zones of roads buffer
buffered_roads['saved_geom'] = buffered_roads.geometry
joined_roads_in_aoi = gpd.sjoin(buffered_roads, buffered_roads[[
                                'OBJECTID', 'OBJEKTART', 'saved_geom', 'geometry']], how='left', lsuffix='1', rsuffix='2')

# Drop excessive rows
intersected = joined_roads_in_aoi[joined_roads_in_aoi['OBJECTID_2'].notna(
)].copy()
intersected_not_itself = intersected[intersected['OBJECTID_1']
                                     != intersected['OBJECTID_2']].copy()
intersected_roads = intersected_not_itself.drop_duplicates(
    subset=['OBJECTID_1', 'OBJECTID_2'])

intersected_roads.reset_index(inplace=True, drop=True)

# Sort the roads so that the widest ones come first
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 0, 'OBJEKTART_1'] = 5.5
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 1, 'OBJEKTART_1'] = 5.6
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 20, 'OBJEKTART_1'] = 8.5
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 21, 'OBJEKTART_1'] = 2.5

intersect_other_width = intersected_roads[intersected_roads['OBJEKTART_1']
                                          < intersected_roads['OBJEKTART_2']].copy()

# WARNING: order set to descending to promote the roads 3m which are a low class. -> otherwise, give a high number to this class...
intersect_other_width.sort_values(
    by=['OBJEKTART_1'], inplace=True, ascending=True)
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 5.5, 'OBJEKTART_1'] = 0
intersected_roads.loc[intersected_roads['OBJEKTART_1']
                      == 5.6, 'OBJEKTART_1'] = 1
intersect_other_width.loc[intersect_other_width['OBJEKTART_1']
                          == 8.5, 'OBJEKTART_1'] = 20
intersect_other_width.loc[intersect_other_width['OBJEKTART_1']
                          == 2.5, 'OBJEKTART_1'] = 21

intersect_other_width.sort_values(
    by=['KUNSTBAUTE'], ascending=False, inplace=True, ignore_index=True)

# Suppress the overlapping intersection
corr_overlap = buffered_roads.copy()

for idx in tqdm(intersect_other_width.index, total=intersect_other_width.shape[0],
                desc='Suppressing the overlap of roads with different width'):

    poly1_id = corr_overlap.index[corr_overlap['OBJECTID'] ==
                                  intersect_other_width.loc[idx, 'OBJECTID_1']].values.astype(int)[0]
    poly2_id = corr_overlap.index[corr_overlap['OBJECTID'] ==
                                  intersect_other_width.loc[idx, 'OBJECTID_2']].values.astype(int)[0]

    corr_overlap = fct_misc.polygons_diff_without_artifacts(
        corr_overlap, poly1_id, poly2_id, keep_everything=True)

corr_overlap.drop(columns=['saved_geom'], inplace=True)
corr_overlap.set_crs(epsg=2056, inplace=True)

# Exclude the roads potentially under forest canopy
print('Suppressing roads under the forest canopy...')
fct_misc.test_crs(corr_overlap.crs, forest.crs)

forest['buffered_geom'] = forest.buffer(3)
forest.drop(columns=['geometry'], inplace=True)
forest.rename(columns={'buffered_geom': 'geometry'}, inplace=True)

non_forest_roads = corr_overlap.copy()
roads_parts = non_forest_roads.overlay(
    forest[['UUID', 'geometry']], how='difference')

roads_parts.drop(columns=['GDB-Code'], inplace=True)
roads_parts.rename(columns={'Width': 'road_width'}, inplace=True)


In [ ]:
roads_parts.to_file(os.path.join(
    PROCESSED_FOLDER, "shapefiles_gpkg/test_clip_MO_TLM.shp"))


In [ ]:
roads_parts.to_crs(crs=roads_domain_union.crs, inplace=True)
fct_misc.test_crs(roads_domain_union, roads_parts)


In [ ]:
roads_parts = fct_misc.test_valid_geom(roads_parts, True, 'road parts')


Cut the roads at intersections


In [ ]:
sectionned_roads = gpd.overlay(
    roads_domain_union, roads_parts, how="intersection")
print(sectionned_roads.columns)


In [ ]:
# sectionned_roads.to_file(os.path.join(PROCESSED_FOLDER, "shapefiles_gpkg/test_sectionned_roads.shp"))

# sectionned_roads_4326 = sectionned_roads.to_crs(epsg=4326)
# sectionned_roads_4326.to_file(os.path.join(PROCESSED_FOLDER, "json/roads_MO_TLM_sectionned.geojson"))

sectionned_roads.to_file(os.path.join(PROCESSED_FOLDER, 'shapefiles_gpkg/roads_MO_TLM.gpkg'), layer="sectionned_roads")


In [ ]:
# added_parts=gpd.overlay(roads_domain_union, roads_parts, how="difference")
# print(added_parts.columns)
# labels=pd.concat([sectionned_roads, added_parts])


Cut the roads by surface type


In [ ]:
roads_parts_surface = gpd.GeoDataFrame()
for surface_type in [100, 200]:
    roads_by_surface = roads_parts[roads_parts['BELAGSART'] == surface_type]

    surface_union_geom = roads_by_surface.unary_union
    surface_union = gpd.GeoDataFrame({'id_suface': [k for k in range(len(surface_union_geom.geoms))],
                                     'geometry': [geom for geom in surface_union_geom.geoms],
                                      'BELAGSART': [surface_type for k in range(len(surface_union_geom.geoms))]},
                                     crs=roads_parts.crs)

    roads_parts_surface = pd.concat(
        [roads_parts_surface, surface_union], ignore_index=True)


In [ ]:
fct_misc.test_crs(roads_domain_union, roads_parts_surface)
sectionned_roads_surface = gpd.overlay(
    roads_domain_union, roads_parts_surface, how="intersection")
print(sectionned_roads_surface.columns)


In [ ]:
# sectionned_roads_surface.to_file(os.path.join(PROCESSED_FOLDER, "shapefiles_gpkg/test_fusionned_surface.shp"))

# sectionned_roads_surface_4326 = sectionned_roads_surface.to_crs(epsg=4326)
# sectionned_roads_surface_4326.to_file(os.path.join(PROCESSED_FOLDER, "json/roads_MO_TLM_sec_by_surface.geojson"))

sectionned_roads_surface.to_file(os.path.join(PROCESSED_FOLDER, 'shapefiles_gpkg/roads_MO_TLM.gpkg'), layer="sectionned_roads_by_surface")


Cut the roads by object type


In [ ]:
roads_parts_type = gpd.GeoDataFrame()
for surface_type in [100, 200]:
    roads_by_surface = roads_parts[roads_parts['BELAGSART'] == surface_type]

    for road_type in roads_by_surface['OBJEKTART'].unique().tolist():
        roads_by_type = roads_by_surface[roads_by_surface['OBJEKTART'] == road_type]

        type_union_geom = roads_by_type.unary_union
        try:
            type_union = gpd.GeoDataFrame({'id_type': [k for k in range(len(type_union_geom.geoms))],
                                          'geometry': [geom for geom in type_union_geom.geoms],
                                           'OBJEKTART': [road_type for k in range(len(type_union_geom.geoms))],
                                           'BELAGSART': [surface_type for k in range(len(type_union_geom.geoms))]},
                                          crs=roads_parts.crs)
        except AttributeError as e:
            if type_union_geom.geom_type == 'Polygon':
                type_union = gpd.GeoDataFrame({'id_type': [0], 'geometry': [type_union_geom],
                                              'OBJEKTART': [road_type], 'BELAGSART': [surface_type]},
                                              crs=roads_parts.crs)
            else:
                print(e)
                sys.exit(1)

        roads_parts_type = pd.concat([roads_parts_type, type_union], ignore_index=True)


In [ ]:
fct_misc.test_crs(roads_domain_union, roads_parts_type)
sectionned_roads_type = gpd.overlay(roads_domain_union, roads_parts_type, how="intersection")
print(sectionned_roads_type.columns)


In [ ]:
# sectionned_roads_type.to_file(os.path.join(PROCESSED_FOLDER, "shapefiles_gpkg/test_fusionned_type.shp"))

# sectionned_roads_type_4326 = sectionned_roads_type.to_crs(epsg=4326)
# sectionned_roads_type_4326.to_file(os.path.join(PROCESSED_FOLDER, "json/roads_MO_TLM_sec_by_type.geojson"))

sectionned_roads_type.to_file(os.path.join(PROCESSED_FOLDER, 'shapefiles_gpkg/roads_MO_TLM.gpkg'), layer="sectionned_roads_by_type")

